In [7]:
import numpy as np
import pprint

In [8]:
instances = ["kroA100.tsp", "kroB100.tsp"]
methods = ["greedy_cycle", "nearest_neighbour", "regret_heuristic"]

In [13]:
records = {}

for instance in instances:
    for method in methods:
        cycle_path = "cycle" + "_" + method + "_" + instance + ".txt"
        routes = []
        routes_length = []
        with open(cycle_path, 'rt') as file:
            lines = file.readlines()

        for i in range(0, len(lines), 2):
            lineA, lineB = [line.strip().split() for line in (lines[i], lines[i+1])]

            id = lineA[0]
            lenA, lenB = lineA[1], lineB[1]
            cycleA, cycleB = lineA[2:], lineB[2:]

            records[id] = {
                "length": [lenA, lenB],
                "cycle": [cycleA, cycleB] }



        # Create a list of tuples, each containing id and the sum of lenA and lenB
        sum_lengths = [(id, int(data["length"][0]) + int(data["length"][1])) for id, data in records.items()]
        sums_array = np.array([item[1] for item in sum_lengths])
        best_index = np.argmin(sums_array)
        best_id = sum_lengths[best_index][0]
        best_cycles = records[best_id]["cycle"]

        sums_array = np.array([item[1] for item in sum_lengths])
        average_sum = np.mean(sums_array)
        min_sum = np.min(sums_array)
        max_sum = np.max(sums_array)

        print(f"Instance: {instance}, Method: {method}")
        # Wypisywanie średniej, minimalnej i maksymalnej sumy długości
        print(f"Średnia suma długości: {average_sum}")
        print(f"Minimalna suma długości: {min_sum}")
        print(f"Maksymalna suma długości: {max_sum}")


        output_file_name = f"cycle_{instance}_{method}_best.txt"
        with open(output_file_name, 'wt') as output_file:
            output_file.write(f"Best ID: {best_id}\n")
            output_file.write(f"Sum of lengths: {sum_lengths[best_index][1]}\n")
            output_file.write("Cycle A: " + ' '.join(best_cycles[0]) + "\n")
            output_file.write("Cycle B: " + ' '.join(best_cycles[1]) + "\n")

Instance: kroA100.tsp, Method: greedy_cycle
Średnia suma długości: 28824.79797979798
Minimalna suma długości: 25716
Maksymalna suma długości: 30605
Instance: kroA100.tsp, Method: nearest_neighbour
Średnia suma długości: 32583.53
Minimalna suma długości: 29394
Maksymalna suma długości: 36018
Instance: kroA100.tsp, Method: regret_heuristic
Średnia suma długości: 32670.93
Minimalna suma długości: 31607
Maksymalna suma długości: 34234
Instance: kroB100.tsp, Method: greedy_cycle
Średnia suma długości: 28516.11
Minimalna suma długości: 25625
Maksymalna suma długości: 33105
Instance: kroB100.tsp, Method: nearest_neighbour
Średnia suma długości: 32246.61
Minimalna suma długości: 27886
Maksymalna suma długości: 35334
Instance: kroB100.tsp, Method: regret_heuristic
Średnia suma długości: 31875.11
Minimalna suma długości: 30616
Maksymalna suma długości: 33923


In [10]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_cycles(data, cycles, filename_suffix):
    plt.figure(figsize=(18, 16))

    for node, (x, y) in data.items():
        plt.scatter(x, y, color='red', s=30)
        plt.text(x, y, str(node), fontsize=8, ha='left', va='bottom')

    colors = ['blue', 'green', 'orange', 'purple', 'cyan']
    for i, cycle in enumerate(cycles):
        cycle_x = [data[node][0] for node in cycle]
        cycle_y = [data[node][1] for node in cycle]
        cycle_x.append(cycle_x[0])
        cycle_y.append(cycle_y[0])
        plt.plot(cycle_x, cycle_y, linestyle='-', color=colors[i % len(colors)], label=f'Cycle {i+1}')

    plt.title(f'Visualization of Cycles for {filename_suffix}')
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')
    plt.grid(True)
    plt.legend()
    plt.savefig(f"{filename_suffix}_visualization.png")
    plt.close() 

if __name__ == "__main__":

    for instance in instances:
        data = {}
        with open(instance, 'r') as file:
            for line in file:
                if line.startswith('EOF'):
                    break
                parts = line.split()
                if parts[0].isdigit():
                    node, x, y = map(int, parts)
                    data[node] = (x, y)

        for method in methods:
            filename = f'cycle_{instance}_{method}_best.txt'
            cycles = []
            try:
                with open(filename, 'r') as file:
                    lines = file.readlines()
                    cycleA = [int(node) for node in lines[2].strip().split()[2:]] # Skipping "Cycle A:" part
                    cycleB = [int(node) for node in lines[3].strip().split()[2:]] # Skipping "Cycle B:" part
                cycles.extend([cycleA, cycleB]) # Add both cycles to the list
                
                filename_suffix = f'{instance}_{method}_best'
                visualize_cycles(data, cycles, filename_suffix)
            except FileNotFoundError:
                print(f"File {filename} not found. Skipping...")

In [11]:
time_stats = {}

for instance in instances:
    for method in methods:
        time_method_path = "time_" + method + "_" + instance + ".txt"
        print(time_method_path)
        times = []
        with open(time_method_path, 'rt') as file:
            for line in file:
                times.append(int(line.split(":")[1]))

        stats = {
            "avg": round(np.mean(times), 2),
            "min": round(np.min(times), 2),
            "max": round(np.max(times), 2)
        }

        time_stats[time_method_path[:-4]] = stats

time_greedy_cycle_kroA100.tsp.txt
time_nearest_neighbour_kroA100.tsp.txt
time_regret_heuristic_kroA100.tsp.txt
time_greedy_cycle_kroB100.tsp.txt
time_nearest_neighbour_kroB100.tsp.txt
time_regret_heuristic_kroB100.tsp.txt


In [12]:
pprint.PrettyPrinter(width=20).pprint(time_stats)

{'time_greedy_cycle_kroA100.tsp': {'avg': 1688.82,
                                   'max': 2453,
                                   'min': 1526},
 'time_greedy_cycle_kroB100.tsp': {'avg': 1730.32,
                                   'max': 3725,
                                   'min': 1609},
 'time_nearest_neighbour_kroA100.tsp': {'avg': 7362.8,
                                        'max': 11392,
                                        'min': 6861},
 'time_nearest_neighbour_kroB100.tsp': {'avg': 7282.62,
                                        'max': 12135,
                                        'min': 6874},
 'time_regret_heuristic_kroA100.tsp': {'avg': 34867.97,
                                       'max': 40296,
                                       'min': 33135},
 'time_regret_heuristic_kroB100.tsp': {'avg': 34982.16,
                                       'max': 47180,
                                       'min': 32262}}
